In [1]:
import pandas as pd
import pandas_datareader as pdr
import datetime
import matplotlib as plt

def get_historical_data(start, end, stock_list):
    for stock in stock_list:
        history = pdr.get_data_yahoo(stock, start, end)
        history.to_csv('data/raw/'+stock+'.csv')

def read_data(stock_list, plot_data):
    stock_data_list = []
    for stock in stock_list:
        stock_data = pd.read_csv('data/raw/'+stock+'.csv', header=0, parse_dates=['Date'])
        #stock_data.set_index('Date',inplace=True)
        if plot_data:
            plott = stock_data['Adj Close'].plot(figsize=(16, 12), label=stock)
            plott.legend()
        stock_data['stock_name'] = stock
        stock_data_list.append(stock_data)
    return stock_data_list

def write_data(stock_data_list):
    for stock_data in stock_data_list:
        file_path = "./data_generated/" + stock_data['stock_name'][0] + '.csv'
        stock_data.to_csv(file_path, sep='\t', encoding='utf-8')
        
        
def read_processed_data(stock_list):
    stock_data_list = []
    for stock in stock_list:
        stock_data = pd.read_csv('data/processed/'+stock+'.csv', header=0, parse_dates=[0])
        stock_data_list.append(stock_data)
    return stock_data_list


In [2]:
def calculate_MACD_RSI(stock_data_list, data_column_name,period1,period2):
    for index, stock_data in enumerate(stock_data_list):
        ema1 = calculate_insert_EMA(stock_data, data_column_name, period1)
        ema2 = calculate_insert_EMA(stock_data, data_column_name, period2)
        macd_column_name = "MACD_" + str(period1) + "_" + str(period2)
        stock_data[macd_column_name] = ema1 - ema2 
        stock_data['sign'] = 0
        stock_data = add_RSI(stock_data)
        stock_data = stock_data[period2:]
        stock_data_list[index] = stock_data
        
    
def calculate_insert_EMA(data, data_column_name, period):
    ema_period = data[data_column_name].ewm(span=period, min_periods=period, adjust= True).mean()
    inserting_column = "ema_" + str(period)
    data[inserting_column] = ema_period
    return data[inserting_column]
    
def null_value_exists(stock_data_list):
    there_exist_null_value = False
    for stock_data in stock_data_list:
        there_exist_null_value = there_exist_null_value or stock_data.isnull().any().any()
    return there_exist_null_value

def detect_sign_change(stock_data_list, column_name):
    for stock_data in stock_data_list:
        for index in range(len(stock_data)-1):
            sign_change = (stock_data[column_name][index]<0) and (stock_data[column_name][index+1]>0) 
            if sign_change:
                stock_data['sign'][index] = 1.0
            else:
                stock_data['sign'][index] = 0.0
                
#Add RSI Wilder's RSI
def add_RSI(df, column='Adj Close', period=14):
    delta = df[column].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    rUp = up.ewm(com=period - 1,  adjust=False).mean()
    rDown = down.ewm(com=period - 1, adjust=False).mean().abs()
    rsi = 100 - 100 / (1 + rUp / rDown)  
    return df.join(rsi.to_frame('RSI'))

In [6]:
stock_list = ['TUPRS.IS', 'EREGL.IS', 'BAGFS.IS', 'EGEEN.IS', 'ULUSE.IS', 'ASELS.IS']




def whole_history():
    start=datetime.datetime(2018,1,1)
    end=datetime.datetime(2020,2,15)

    get_historical_data(start,end,stock_list)
    stock_data_list = read_data(stock_list,False)

    calculate_MACD_RSI(stock_data_list, "Adj Close",9,21)
    print(stock_data_list[0])
    detect_sign_change(stock_data_list,"MACD_9_21")

    #stock_data_list[0].tail()
    #print(stock_data_list[0])

    write_data(stock_data_list)


    
whole_history()

KeyError: 0

In [ ]:
#differential work

# read data
# read last day data available
# compare with yesterday and if it is different get new data
# add ema 9, ema 21 macd 9, 21 , rsi
# list today sign changes
# list today rsi changes

from datetime import date

def differential():
    stock_data_list = read_processed_data(stock_list)
    last_data_date = stock_data_list[1].iloc[-1]["Date"]
    last_data_date = datetime.datetime.strptime(str(last_data_date), '%Y-%m-%d %H:%M:%S')
    last_data_date = last_data_date.date()
    today = datetime.date.today()
    diff = today - last_data_date
    diff = diff.days
    if(diff>1):
        get_historical_data(last_data_date+1, today, stock_list,)
        raw_datalist = read_data(stock_list, False)
        for index, stock_data in iter(stock_data_list):
            stock_data.append(raw_datalist[index])
            print(stock_data)
        
            
        
differential()
